In [17]:
import pandas as pd
import numpy as np

In [18]:
pathways = pd.read_csv("/idiap/group/genomics/annotation/KEGG/human/KEGG_human_agg.csv", index_col=0)
pathways['Genes'] = pathways['Gene_Symbols'].apply(lambda x: x.split(', '))
pathways.drop(columns=['Gene_Symbols', 'Symbol'], inplace=True)

In [21]:
data = pd.read_csv("/idiap/group/genomics/lfournier/digitalhistopathology/results/molecular/filtered_normalized_gene_expression.csv", index_col=0)

In [45]:
def process_one_spot_fast(spot, gene_list, n_bootstrap=1000):
    # Filter valid genes
    genes_in_data = [gene for gene in gene_list if gene in spot.index]
    gene_count = len(genes_in_data)

    # If no genes matched, return NaN or another placeholder
    if gene_count == 0:
        return np.nan

    # Convert to NumPy for speed
    spot_values = spot.values
    spot_genes = np.array(spot.index)
    
    # Get indices for target genes
    gene_idx = np.isin(spot_genes, genes_in_data)
    observed_mean = spot_values[gene_idx].mean()

    # Generate bootstrap samples: (n_bootstrap × gene_count) indices
    sampled_indices = np.random.choice(len(spot), size=(n_bootstrap, gene_count), replace=True)

    # Gather values and compute means (vectorized)
    sampled_values = spot_values[sampled_indices]
    bootstrapped_means = sampled_values.mean(axis=1)

    # Compute two-sided p-value
    boot_mean = bootstrapped_means.mean()
    obs_diff = np.abs(observed_mean - boot_mean)
    extreme_count = np.sum(np.abs(bootstrapped_means - boot_mean) >= obs_diff)
    p_value = (extreme_count + 1) / (n_bootstrap + 1)

    # Return signed log10 p-value
    return -np.log10(p_value) if observed_mean > boot_mean else np.log10(p_value)

In [ ]:
import pandas as pd
import numpy as np
import multiprocessing as mp
from tqdm import tqdm

# Transpose to spots x genes
data_T = data.T  # shape: (n_spots, n_genes)
spot_names = data_T.index
pathway_names = pathways.index

# Shared global data (only needed if using starmap or apply_async)
global_data_T = data_T

# Function that computes for one pathway
def compute_pathway(args):
    pathway_name, gene_list = args
    result_series = global_data_T.apply(lambda spot: process_one_spot_fast(spot, gene_list), axis=1)
    result_series.name = pathway_name  # helpful for final combination
    return result_series

# Prepare list of (name, gene_list) for each pathway
pathway_args = [(name, pathways.loc[name, 'Genes']) for name in pathway_names]

# Use multiprocessing with tqdm
with mp.Pool(processes=mp.cpu_count()) as pool:
    results = list(tqdm(pool.imap(compute_pathway, pathway_args), total=len(pathway_args)))
    
results_df = pd.concat(results, axis=1)
results_df = results_df.loc[spot_names, pathway_names]  # ensure proper order

results_df.to_csv("pathway_enrichment_results.csv")

100%|██████████| 2/2 [00:46<00:00, 23.27s/it]


In [72]:
pathways.index

Index(['2-Oxocarboxylic acid metabolism ', 'ABC transporters ',
       'AGE-RAGE signaling pathway in diabetic complications ',
       'AMPK signaling pathway ', 'ATP-dependent chromatin remodeling ',
       'Acute myeloid leukemia ', 'Adherens junction ',
       'Adipocytokine signaling pathway ',
       'Adrenergic signaling in cardiomyocytes ', 'African trypanosomiasis ',
       ...
       'Vitamin digestion and absorption ', 'Wnt signaling pathway ',
       'Yersinia infection ', 'alpha-Linolenic acid metabolism ',
       'beta-Alanine metabolism ', 'cAMP signaling pathway ',
       'cGMP-PKG signaling pathway ', 'mRNA surveillance pathway ',
       'mTOR signaling pathway ', 'p53 signaling pathway '],
      dtype='object', name='Description', length=357)

In [33]:
data['B-rep1-spot10x13']

A2M       0.000000
A4GALT    0.000000
AAAS      0.000000
AACS      0.000000
AAED1     0.000000
            ...   
ZXDC      0.000000
ZYG11B    0.000000
ZYX       3.307465
ZZEF1     0.000000
ZZZ3      0.000000
Name: B-rep1-spot10x13, Length: 11591, dtype: float64

In [36]:
process_one_spot(data['B-rep1-spot10x13'], pathways.iloc[0]['Genes'])

0.9035240644712622

In [35]:
np.log10(0.05)

-1.3010299956639813